In [ ]:
from awpy.parser import DemoParser
from awpy.visualization.plot import plot_round
from dataclasses import dataclass, field, fields
from dataclasses_json import dataclass_json
import csv

In [ ]:
demo_file_name = r"demos/cloud9-vs-outsiders-m1-dust2.dem"

p = DemoParser(demofile=demo_file_name, parse_rate=128, parse_frames=True)

data = p.parse()

In [ ]:
@dataclass_json
@dataclass
class Position:
    """
    x, y, and z coordinates in a CSGO map.
    """
    x: float
    y: float
    z: float

@dataclass_json
@dataclass
class PlayerInfo:
    """
    Contains information about a CSGO player including their name, their team, and a list of their positions throughout the gme
    """
    name: str
    team: str
    positions: list[Position] # Length of this list is the number of frames in the round

@dataclass_json
@dataclass
class RoundInfo:
    """
    Contains a list of PlayerInfo objects, one for each player in the CSGO game round.
    """
    players: list[PlayerInfo] = field(default_factory=list)# Length of this list is the number of rounds in the game

@dataclass_json
@dataclass
class Game:
    """
    Contains a list of RoundInfo objects, one for each round in the CSGO game.
    """
    round_info: list[RoundInfo] = field(default_factory=list) # Length of this list is the number of rounds in the game

    def add_player_info(self, round_index: int, player_name: str, team:str, position: Position):
        """
        Uses the given information to construct a PlayerInfo object and add it to the RoundInfo object indexed by round_index
        """
        while len(self.round_info) <= round_index:
            self.round_info.append(RoundInfo())
        for player in self.round_info[round_index].players:
            if player.name == player_name:
                player.positions.append(position)
                return
        self.round_info[round_index].players.append(PlayerInfo(player_name, team, [position]))
        return

game = Game()

@dataclass_json
@dataclass
class PlayerState:
    """
    """
    round_index: int
    player_name: str
    team: str
    x: float
    y: float
    z: float

flat_game_info: list[PlayerState] = []

for game_round_index in range(0, len(data["gameRounds"])):
    frames = data["gameRounds"][game_round_index]["frames"]
    # print("Round: ", game_round_index + 1)
    for index, frame in enumerate(frames):
        # print("Frame: ", index + 1)
        for team in ["ct", "t"]:
            for player in frame[team]["players"]:
                position = Position(player["x"], player["y"], player["z"])
                # print(player["name"], position)
                game.add_player_info(game_round_index, player["name"], team, position)
                flat_game_info.append(PlayerState(game_round_index, player["name"], team, player["x"], player["y"], player["z"]))

def debug_print_game(game: Game):
    print("Round count: ", len(game.round_info))
    for round in game.round_info:
        print("Player count: ", len(round.players))
        for player in round.players:
            print("Frame count: ", len(player.positions))

# print(game.to_json())
print(flat_game_info)



In [ ]:
with open("data.csv", "w") as output_file:
    writer = csv.writer(output_file)
    writer.writerow([class_field.name for class_field in fields(PlayerState)])
    for state_entry in flat_game_info:
        writer.writerow([getattr(state_entry, field_name) for field_name in [class_field.name for class_field in fields(PlayerState)]])

In [ ]:
d = data
plot_round("best_round_ever.gif", d["gameRounds"][7]["frames"], map_name=d["mapName"], map_type="simpleradar", dark=False)

In [ ]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
# print(data.keys())

In [ ]:
parser_params = data["parserParameters"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

In [ ]:
parser_params = data["serverVars"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

In [ ]:
parser_params = data["matchPhases"]
for key in parser_params.keys():
    print(f"{key}: {parser_params[key]}")

In [ ]:
rounds = data["gameRounds"]
first_round = rounds[0]
for key in first_round.keys():
    print(key, type(first_round[key]))

In [ ]:
frames = first_round["frames"]
first_frame = frames[0]
for key in first_frame.keys():
    print(key, type(first_frame[key]))

In [1]:
# Loading demo from .json
import models
import json

with open("cloud9-vs-outsiders-m1-dust2.json", "r") as file:
    data = json.load(file)

demo = models.deserialize_demo_data(data)

# Creating timeline and events from demo
import timelining

timeline = timelining.create_timeline(demo)
# print(len(timeline.events))
# for event in timeline.events:
#     print(event)

d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in double_scalars
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi
d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in arccos
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi


In [2]:
import timelining

round_boundary_ticks: list[int] = [(round.frames[0].tick, round.frames[-1].tick) for round in demo.game_rounds]

round_frame_ticks: list[list[int]] = []
for round in demo.game_rounds:
    round_frame_ticks.append([frame.tick for frame in round.frames])

organized_events: list[timelining.Event] = [[] for _ in range(0, len(demo.game_rounds))]
for event in timeline.events:
    for frame_index, frame_ticks in enumerate(round_frame_ticks):
        if event.tick in frame_ticks:
            organized_events[frame_index].append(event)
            break
# print(len(organized_events))
# print([len(frame) for frame in organized_events])

import plotting
for round in demo.game_rounds[0:1]:
    plotting.plot_round(f"round_{round.round_number}.gif", round.frames, map_name=demo.map_name, map_type="simpleradar", dark=False, show_tiles=True)

# TODO: Organize events into frames so each frame of the GIF can have the associated events grouped with it


Drawing frames:  40%|███▉      | 38/96 [01:41<02:52,  2.98s/it]

In [ ]:
# import plotting

# plotting.plot_tiles(map_name = "de_dust2")

In [ ]:
# Writing events to csv

import csv
from dataclasses import fields

with open("events.csv", "w") as file:
    writer = csv.writer(file)
    all_fields: list[str] = []
    for event in timeline.events:
        all_fields.extend([field.name for field in fields(event)])
    unique_fields = list(dict.fromkeys(all_fields))
    # print(unique_fields)
    writer.writerow([field for field in unique_fields])
    for event in timeline.events:
        writer.writerow([getattr(event, field, None) for field in unique_fields])

# Writing event descriptions to .csv

import csv
from dataclasses import fields

with open("event_descriptions.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["tick", "name", "description"])
    for event in timeline.events:
        writer.writerow([event.tick, event.__class__.__name__, str(event)])